This notebook is for fine tuning the model and training

In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset, Dataset, concatenate_datasets
import pandas as pd

In [ ]:
# Define paths
DATA_DIR = "data"
MODEL_DIR = "models/t5_myth_buster"
PROCESSED_DATA_PATH = os.path.join(DATA_DIR, "ai_myths_expanded_1000_complexity.json")

# Load Processed Dataset
dataset = load_dataset("json", data_files=PROCESSED_DATA_PATH)["train"]

## Load Pre-trained T5 Model & Tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base", model_max_length=128)
model = T5ForConditionalGeneration.from_pretrained("t5-base")

## Tokenize Data
def preprocess_function(examples):
    inputs = ["debunk: " + myth for myth in examples["Myth"]]
    targets = [fact for fact in examples["Debunked Fact"]]

    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length").input_ids

    model_inputs["labels"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels
    ]

    return model_inputs


# Apply tokenization
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Split the tokenized dataset into Simple and Complex subsets.
simple_dataset = tokenized_datasets.select(range(500))
complex_dataset = tokenized_datasets.select(range(500, 1000))


In [ ]:

# Stage 1: Fine-tune on the simple dataset
training_args_stage1 = TrainingArguments(
    output_dir="./results_stage1",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    num_train_epochs=7,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs_stage1",
    report_to="none",
)
trainer_stage1 = Trainer(
    model=model,
    args=training_args_stage1,
    train_dataset=simple_dataset,
    eval_dataset=simple_dataset,
    tokenizer=tokenizer,
)
trainer_stage1.train()

# Stage 2: Fine-tune on a combined dataset (Simple + Complex)

combined_dataset = concatenate_datasets([simple_dataset, complex_dataset])
training_args_stage2 = TrainingArguments(
    output_dir="./results_stage2",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    num_train_epochs=7,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs_stage2",
    report_to="none",
)
trainer_stage2 = Trainer(
    model=model,
    args=training_args_stage2,
    train_dataset=combined_dataset,
    eval_dataset=combined_dataset,
    tokenizer=tokenizer,
)
trainer_stage2.train()

In [ ]:
# Save final model & tokenizer
model.save_pretrained(MODEL_DIR)
tokenizer.save_pretrained(MODEL_DIR)
print(f"Model saved at {MODEL_DIR}")